# 🔤 T5 Encoder + Neural Network for Price Classification - Amazon ML Challenge 2025

## Encoder-Only Transformer Approach with Classification

This notebook implements a **T5 Encoder-based classification model** for product price prediction:

### Architecture Overview:
1. **T5 Text Encoder**
   - Uses T5EncoderModel for text understanding
   - Pre-trained transformer encoder (no decoder)
   - Generates rich contextual representations
   - Efficient compared to full T5 model

2. **Price Binning Strategy**
   - Converts regression to classification problem
   - Creates price bins/ranges for better learning
   - Reduces complexity of direct price prediction
   - Handles price distribution challenges

3. **Neural Network Classifier**
   - Multi-layer perceptron on top of T5 embeddings
   - Dropout and batch normalization
   - Softmax output for price bin probabilities
   - Cross-entropy loss for classification

### Training Strategy:
- **Transfer Learning**: Leverage T5's pre-trained knowledge
- **Fine-tuning**: Adapt encoder for price prediction task
- **Batch Processing**: Efficient GPU utilization
- **Class Balancing**: Handle imbalanced price distributions

### Key Features:
- **Serverless Training**: Runs on Modal cloud infrastructure
- **Memory Efficient**: Encoder-only approach
- **Robust Classification**: Handles price ranges well
- **Scalable**: Easy to deploy and scale

### Advantages:
- Simpler than full generative models
- Good performance on classification tasks
- Fast inference for production use
- Cloud-native development environment

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5EncoderModel
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np

batch_size = 32

# -------------------------------------------------
# 1. Load and Clean Data
# -------------------------------------------------
df = pd.read_csv('/root/final_data_with_catalog.csv')

# Check for missing values
print(f"Total rows: {len(df)}")
print(f"Missing price_bin values: {df['price_bin'].isna().sum()}")
print(f"Missing catalog_content values: {df['catalog_content'].isna().sum()}")

# Drop rows with missing price_bin or catalog_content
df = df.dropna(subset=['price_bin', 'catalog_content'])
print(f"Rows after dropping NaN: {len(df)}")

# Convert price_bin intervals to integer indices
# Create a mapping from unique intervals to integer indices
unique_bins = sorted(df['price_bin'].unique())
bin_to_idx = {bin_val: idx for idx, bin_val in enumerate(unique_bins)}
df['price_bin'] = df['price_bin'].map(bin_to_idx)

print(f"Number of unique price bins: {df['price_bin'].nunique()}")
print(f"Price bin range: {df['price_bin'].min()} to {df['price_bin'].max()}")

# Check class distribution
bin_counts = df['price_bin'].value_counts().sort_index()
print(f"\nClass distribution:")
print(bin_counts)

# Filter out bins with less than 2 samples (required for stratified split)
min_samples_per_bin = 2
valid_bins = bin_counts[bin_counts >= min_samples_per_bin].index
df = df[df['price_bin'].isin(valid_bins)]

print(f"\nRows after filtering rare bins: {len(df)}")
print(f"Remaining unique bins: {df['price_bin'].nunique()}")

# Remap price_bin to consecutive integers after filtering
unique_bins_filtered = sorted(df['price_bin'].unique())
bin_remap = {old_idx: new_idx for new_idx, old_idx in enumerate(unique_bins_filtered)}
df['price_bin'] = df['price_bin'].map(bin_remap)

print(f"Final price bin range: {df['price_bin'].min()} to {df['price_bin'].max()}")

# Train-test split with stratification
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['price_bin'])

# -------------------------------------------------
# 2. Tokenizer and T5 Encoder
# -------------------------------------------------
tokenizer = T5Tokenizer.from_pretrained("t5-base")
t5_encoder = T5EncoderModel.from_pretrained("t5-base")

# Freeze T5 (optional if you don't want fine-tuning)
for param in t5_encoder.parameters():
    param.requires_grad = False

# -------------------------------------------------
# 3. Dataset Class
# -------------------------------------------------
class TextBinDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=105):
        self.texts = df['catalog_content'].tolist()
        self.labels = df['price_bin'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        enc = self.tokenizer(
            text, padding='max_length', truncation=True,
            max_length=self.max_len, return_tensors='pt'
        )
        return {
            'input_ids': enc['input_ids'].squeeze(0),
            'attention_mask': enc['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

# -------------------------------------------------
# 4. Model Definition
# -------------------------------------------------
class T5MLPClassifier(nn.Module):
    def __init__(self, t5_model, hidden_dim=512, num_classes=50):
        super().__init__()
        self.t5 = t5_model
        self.mlp = nn.Sequential(
            nn.Linear(768, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.t5(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state.mean(dim=1)  # mean pooling
        logits = self.mlp(embeddings)
        return logits

# -------------------------------------------------
# 5. Dataloaders
# -------------------------------------------------
train_dataset = TextBinDataset(train_df, tokenizer)
val_dataset = TextBinDataset(val_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# -------------------------------------------------
# 6. Training Setup
# -------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5MLPClassifier(t5_encoder, hidden_dim=512, num_classes=df['price_bin'].nunique()).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.mlp.parameters(), lr=1e-4)

# -------------------------------------------------
# 7. Training Loop
# -------------------------------------------------
def evaluate(model, dataloader):
    """Compute loss, accuracy, and MAD on validation set."""
    model.eval()
    val_loss, correct, total = 0, 0, 0
    preds_all, labels_all = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            val_loss += loss.item() * labels.size(0)

            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            preds_all.extend(preds.cpu().numpy())
            labels_all.extend(labels.cpu().numpy())

    val_acc = correct / total
    mad = np.abs(np.array(preds_all) - np.array(labels_all)).mean()
    avg_loss = val_loss / total
    return avg_loss, val_acc, mad


epochs = 60
for epoch in range(epochs):
    model.train()
    total_loss, total_correct, total_samples = 0, 0, 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * labels.size(0)
        preds = torch.argmax(logits, dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

    train_acc = total_correct / total_samples
    train_loss = total_loss / total_samples

    # Run validation at the end of every epoch
    val_loss, val_acc, val_mad = evaluate(model, val_loader)

    print(f"\nEpoch {epoch+1} Summary:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val   Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | Val MAD: {val_mad:.4f}")
    print("-" * 60)

Total rows: 75000
Missing price_bin values: 0
Missing catalog_content values: 0
Rows after dropping NaN: 75000
Number of unique price bins: 54
Price bin range: 0 to 53

Class distribution:
price_bin
0      738
1      746
2     1506
3     1505
4     1147
5     1860
6      750
7     1481
8     1473
9     1554
10    1497
11     739
12    1449
13    1540
14    1497
15    1498
16    1510
17    1494
18    1500
19    1498
20    1335
21    1668
22    1426
23    1575
24    1500
25    1500
26    1515
27    1446
28    1477
29    1500
30    1498
31    1500
32    1501
33    1576
34    1491
35    1497
36    1504
37    1502
38    1174
39    1818
40    1485
41    1491
42    1244
43    1769
44       1
45      10
46    1519
47    1133
48    1865
49    1505
50    1505
51    1495
52    1502
53    1491
Name: count, dtype: int64

Rows after filtering rare bins: 74999
Remaining unique bins: 53
Final price bin range: 0 to 52


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

Epoch 6/60 [Train]: 100%|██████████████████████████████████████████████████| 1875/1875 [05:45<00:00,  5.42it/s]



Epoch 6 Summary:
  Train Loss: 1.9669 | Train Acc: 0.3795
  Val   Loss: 1.6641 | Val Acc: 0.4795 | Val MAD: 6.7883
------------------------------------------------------------


Epoch 7/60 [Train]: 100%|██████████████████████████████████████████████████| 1875/1875 [05:40<00:00,  5.50it/s]



Epoch 7 Summary:
  Train Loss: 1.8325 | Train Acc: 0.4180
  Val   Loss: 1.5334 | Val Acc: 0.5110 | Val MAD: 6.3609
------------------------------------------------------------


Epoch 8/60 [Train]:  89%|████████████████████████████████████████████▍     | 1668/1875 [05:02<00:37,  5.50it/s]